<center><a href="https://sites.google.com/fat.uerj.br/livia/"> <img src="../images/capa2.png" alt="Header" style="width: 800px;"/> </a></center>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vnikoofard/DeepLearningTF/blob/main/notebooks/02_asl.ipynb)

# Classificação de imagem para um conjunto de dados de língua de sinais americana

Nesta seção, executaremos as etapas de preparação de dados, criação de modelo e treinamento de modelo que observamos na última seção usando um conjunto de dados diferente: imagens de mãos fazendo letras em [American Sign Language](http://www.asl.gs).

## Objetivos

* Preparar dados de imagem para treinamento
* Crie e compile um modelo simples para classificação de imagens
* Treine um modelo de classificação de imagem e observe os resultados

## Conjunto de dados da língua de sinais americana

O [alfabeto da linguagem de sinais americana](http://www.asl.gs/) contém 26 letras. Duas dessas letras (j e z) requerem movimento, portanto, não são incluídas no conjunto de dados de treinamento.

<img src="../images/asl.png" style="width: 600px;">

### Kaggle

Este conjunto de dados está disponível no site [Kaggle](http://www.kaggle.com), que é um lugar fantástico para encontrar conjuntos de dados e outros recursos de aprendizado profundo. Além de fornecer recursos como conjuntos de dados e "kernels" que são como esses notebooks, o Kaggle organiza competições das quais você pode participar, competindo com outras pessoas no treinamento de modelos de alta precisão.

Se você deseja praticar ou ver exemplos de muitos projetos de aprendizado profundo, o Kaggle é um ótimo site para visitar.

## Carregando os dados

Este conjunto de dados não está disponível via Keras da mesma forma que o MNIST, então vamos aprender como carregar dados personalizados. No final desta seção, teremos as variáveis `x_train`, `y_train`, `x_valid` e `y_valid` como antes.

### Lendo nos Dados

O conjunto de dados da língua de sinais está no formato [CSV](https://en.wikipedia.org/wiki/Comma-separated_values) (valores separados por vírgula), a mesma estrutura de dados por trás do Microsoft Excel e do Planilhas Google. É uma grade de linhas e colunas com rótulos na parte superior, como visto nos conjuntos de dados [train](../data/asl_data/sign_mnist_train.csv) e [valid](../data/asl_data/sign_mnist_valid.csv) (eles podem demorar um pouco para carregar).

Para carregar e trabalhar com os dados, usaremos uma biblioteca chamada [Pandas](https://pandas.pydata.org/), que é uma ferramenta de alto desempenho para carregar e manipular dados. Vamos ler os arquivos CSV em um formato chamado [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html).

In [1]:
import pandas as pd

Pandas tem um método [read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) que espera um arquivo csv e retorna um DataFrame:

In [3]:
train_url = "https://github.com/vnikoofard/DeepLearningTF/raw/main/data/asl_data/sign_mnist_train.csv"
valid_url = "https://github.com/vnikoofard/DeepLearningTF/raw/main/data/asl_data/sign_mnist_valid.csv"

train_df = pd.read_csv(train_url)
valid_df = pd.read_csv(valid_url)

### Explorando os dados

Vamos dar uma olhada em nossos dados. Podemos usar o método [head](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html) para imprimir as primeiras linhas do DataFrame. Cada linha é uma imagem que possui uma coluna `label` e também 784 valores representando cada valor de pixel na imagem, assim como no conjunto de dados MNIST. Observe que os rótulos atualmente são valores numéricos, não letras do alfabeto:

In [ ]:
train_df.head()

### Extraindo os rótulos

Assim como no MNIST, gostaríamos de armazenar nossos rótulos de treinamento e validação nas variáveis `y_train` e `y_valid`. Aqui criamos essas variáveis e, em seguida, excluímos os rótulos de nossos dataframes originais, onde não são mais necessários:

In [ ]:
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

### Extraindo as Imagens

Assim como no MNIST, gostaríamos de armazenar nossas imagens de treinamento e validação nas variáveis `x_train` e `x_valid`. Aqui criamos essas variáveis:

In [ ]:
x_train = train_df.values
x_valid = valid_df.values

### Resumindo os dados de treinamento e validação

Agora temos 27.455 imagens com 784 pixels cada para treinamento...

In [ ]:
x_train.shape

...bem como seus rótulos correspondentes:

In [ ]:
y_train.shape

Para validação, temos 7.172 imagens...

In [ ]:
x_valid.shape

...e seus rótulos correspondentes:

In [ ]:
y_valid.shape

## Visualizando os Dados

Para visualizar as imagens, usaremos novamente a biblioteca matplotlib. Não precisamos nos preocupar com os detalhes dessa visualização, mas se estiver interessado, você pode aprender mais sobre [matplotlib](https://matplotlib.org/) posteriormente.

Observe que teremos que remodelar os dados de sua forma 1D atual de 784 pixels, para uma forma 2D de 28x28 pixels para dar sentido à imagem:

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(40,40))

num_images = 20
for i in range(num_images):
    row = x_train[i]
    label = y_train[i]
    
    image = row.reshape(28,28)
    plt.subplot(1, num_images, i+1)
    plt.title(label, fontdict={'fontsize': 30})
    plt.axis('off')
    plt.imshow(image, cmap='gray')

## Exercício: Normalize os dados da imagem

Como fizemos com o conjunto de dados MNIST, vamos normalizar os dados da imagem, o que significa que seus valores de pixel, em vez de estarem entre 0 e 255 como estão atualmente:

In [ ]:
x_train.min()

In [ ]:
x_train.max()

...devem ser valores de ponto flutuante entre 0 e 1. Use a seguinte célula para trabalhar. Se você ficar preso, veja a solução abaixo.

In [ ]:
# TODO: Normalize x_train e x_valid.

### Solution

Clique no '...' abaixo para mostrar a solução.

In [ ]:
# SOLUTION
x_train = x_train / 255
x_valid = x_valid / 255

## Exercício: Categorizar os rótulos

Como fizemos com o conjunto de dados MNIST, vamos codificar categoricamente os rótulos. Lembre-se de que podemos usar o método [keras.utils.to_categorical](https://www.tensorflow.org/api_docs/python/tf/keras/utils/to_categorical) para fazer isso passando os valores a serem codificados e, o número de categorias para codificá-lo. Faça o seu trabalho na célula abaixo. Importamos `keras` e definimos o número de categorias (24) para você.

In [ ]:
import tensorflow.keras as keras
num_classes = 24

In [ ]:
# TODO: Categorically encode y_train and y_valid.

### Solução

Clique no '...' abaixo para mostrar a solução.

In [ ]:
# SOLUTION
y_train = keras.utils.to_categorical(y_train, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

## Exercício: Construir o Modelo

Os dados estão todos preparados, temos imagens normalizadas para treinamento e validação, bem como rótulos codificados categoricamente para treinamento e validação.

Para este exercício vamos construir um modelo sequencial. Assim como da última vez, construa um modelo que:
* Tem uma camada de entrada densa. Essa camada deve conter 512 neurônios, usar a função de ativação `relu` e esperar imagens de entrada com uma forma de `(784,)`
* Possui uma segunda camada densa com 512 neurônios que usa a função de ativação `relu`
* Tem uma camada de saída densa com neurônios igual ao número de classes, usando a função de ativação `softmax`

Faça seu trabalho na célula abaixo, criando uma variável `model` para armazenar o modelo. Importamos a classe de modelo Keras [Sequental](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential) e [Dense](https://www.tensorflow.org/api_docs/python /tf/keras/layers/Dense) para você começar. Revele a solução abaixo para uma dica:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# TODO: construa um modelo seguindo as orientações acima.

### Solução

Clique no '...' abaixo para mostrar a solução.

In [ ]:
# SOLUTION
model = Sequential()
model.add(Dense(units = 512, activation='relu', input_shape=(784,)))
model.add(Dense(units = 512, activation='relu'))
model.add(Dense(units = num_classes, activation='softmax'))

## Resumindo o Modelo

Execute a célula abaixo para resumir o modelo que você acabou de criar:

In [ ]:
model.summary()

## Compilando o Modelo

Iremos [compilar](https://www.tensorflow.org/api_docs/python/tf/keras/Sequential#compile) nosso modelo com as mesmas opções de antes, usando [categorical crossentropy](https://www. tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy) para refletir o fato de que queremos nos encaixar em uma das muitas categorias e medir a precisão do nosso modelo:

In [ ]:
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])

## Exercício: Treine o Modelo

Use o método `fit` do modelo para treiná-lo por 20 épocas usando as imagens e rótulos de treinamento e validação criados acima:

In [ ]:
# TODO: Treine o modelo por 20 epochs

### Solução

Clique no '...' abaixo para mostrar a solução.

In [ ]:
# SOLUTION
model.fit(x_train, y_train, epochs=20, verbose=1, validation_data=(x_valid, y_valid))

## Discussão: O que aconteceu?

Podemos ver que a precisão do treinamento atingiu um nível bastante alto, mas a precisão da validação não foi tão alta. O que aconteceu aqui?

Pense um pouco antes de clicar no '...' abaixo para revelar a resposta.

`#SOLUÇÃO`
Este é um exemplo do modelo aprendendo a categorizar os dados de treinamento, mas com desempenho ruim em relação a novos dados nos quais não foi treinado. Essencialmente, ele está memorizando o conjunto de dados, mas não obtendo uma compreensão robusta e geral do problema. Este é um problema comum chamado *overfitting*. Discutiremos o overfitting nas próximas duas palestras, bem como algumas maneiras de abordá-lo.

## Resumo

Nesta seção, você construiu sua própria rede neural para realizar uma classificação de imagem bastante precisa. Parabéns!

Neste ponto, devemos estar um pouco familiarizados com o processo de carregamento de dados (incluindo rótulos), preparando-os, criando um modelo e, em seguida, treinando o modelo com dados preparados.

### Resetar o ambiente
Antes de prosseguir, execute a célula a seguir para resetar o ambiente. Isso é necessário para passar para o próximo notebook.

In [ ]:
import os
os._exit(00)

## Next

Agora que você construiu alguns modelos bastante básicos e eficazes, começaremos a aprender sobre modelos mais sofisticados, incluindo *Redes Neurais Convolucionais* (*Convolutional Neural Networks*).